In [1]:
%pip install transformers

     ---------------------------------------- 8.4/8.4 MB 825.1 kB/s eta 0:00:00
     ------------------------------------ 269.7/269.7 kB 828.8 kB/s eta 0:00:00
     ------------------------------------ 330.1/330.1 kB 639.7 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 586.2 kB/s eta 0:00:00
     ------------------------------------ 169.0/169.0 kB 407.0 kB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [2]:
from transformers import ChatGPT

# Instantiate the ChatGPT model
model = ChatGPT.from_pretrained("facebook/chatbot-gpt-2.0")

# Define a function to generate responses
def generate_response(prompt):
    conversation_history = model.generate_response(prompt)
    return conversation_history.choices[0].message['content']

# Example conversation starter
user_input = "Can you tell me about the company culture?"

# Get chatbot's response
bot_response = generate_response(user_input)

# Print the bot's response
print(bot_response)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ImportError: cannot import name 'ChatGPT' from 'transformers' (c:\Users\Shrinjita Paul\anaconda3\lib\site-packages\transformers\__init__.py)

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import Conversation, InputExample
from transformers import Trainer, TrainingArguments

# Define your conversational dataset
conversations = [
    Conversation(evidence=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you tell me about the company culture?"}
    ]),
    # Add more conversations as needed
]

# Convert the conversations to InputExamples
examples = [InputExample.from_conversation(conv) for conv in conversations]

# Tokenizer and model initialization
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel(GPT2Config.from_pretrained("gpt2"))

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./fine_tuned_chatbot",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=examples,
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_chatbot")

# Now you can load and use the fine-tuned model for generating responses
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_chatbot")

# Example usage
user_input = "Can you tell me about the company culture?"
input_ids = tokenizer.encode(user_input, return_tensors="pt")
bot_response_ids = fine_tuned_model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2)
bot_response = tokenizer.decode(bot_response_ids[0], skip_special_tokens=True)
print(bot_response)


AttributeError: type object 'InputExample' has no attribute 'from_conversation'

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, Conversation

# Define your conversational dataset
conversations = [
    Conversation(evidence=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you tell me about the company culture?"}
    ]),
    # Add more conversations as needed
]

# Tokenizer and model initialization
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel(GPT2Config.from_pretrained("gpt2"))

# Tokenize the conversations
inputs = tokenizer([conv.to_dict() for conv in conversations], return_tensors="pt", padding=True, truncation=True)

# Fine-tune the model
labels = inputs["input_ids"].clone()
labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding tokens during training

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):  # Adjust the number of epochs as needed
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_chatbot")

# Now you can load and use the fine-tuned model for generating responses
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_chatbot")

# Example usage
user_input = "Can you tell me about the company culture?"
input_ids = tokenizer.encode(user_input, return_tensors="pt")
bot_response_ids = fine_tuned_model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2)
bot_response = tokenizer.decode(bot_response_ids[0], skip_special_tokens=True)
print(bot_response)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

c:\Users\Shrinjita Paul\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Shrinjita Paul\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

ImportError: 
GPT2LMHeadModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
